## 8, 9 월 데이터로 학습 / 10월 데이터로 모델 평가 수행

### 목차
1. User 군집 설정하기
2. 필요 함수 생성
3. LightFM_ver1
    - 모델 코드
    - 적용 예시
4. LightFM_ver2
    - 모델 코드
    - 적용 예시
5. SVD
    - 모델 코드
    - 적용 예시
6. 모델별 TEST 수행
7. 최종 결과 도출

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = 'NanumSquare'

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [ ]:
users = pd.read_csv('/content/drive/MyDrive/DXSCHOOL/Project/data_final/db_userinfo.csv')
vods = pd.read_csv('/content/drive/MyDrive/DXSCHOOL/Project/data_final/db_vods_sumut.csv',dtype={'program_id':str})
conts = pd.read_csv('/content/drive/MyDrive/DXSCHOOL/Project/data_final/db_contlog.csv',dtype={'program_id':str})
program_info_all = pd.read_csv('/content/drive/MyDrive/DXSCHOOL/Project/data_final/db_vodinfo.csv',dtype={'program_id':str})

In [ ]:
# e_bool == 0 인 데이터만 뽑기
vod_log = vods[vods['e_bool']==0][['subsr_id', 'program_id', 'program_name', 'episode_num', 'log_dt', 'use_tms', 'disp_rtm_sec', 'count_watch', 'month', 'day', 'age_limit']]
cont_log = conts[conts['e_bool']==0][['subsr_id', 'program_id', 'program_name', 'episode_num', 'log_dt', 'month', 'day', 'age_limit']]
vod_info = program_info_all[program_info_all['e_bool']==0][['program_id','program_name', 'ct_cl', 'program_genre', 'release_date', 'age_limit']]
user_info = users.copy()

### User 군집 설정하기 (8,9월 데이터를 바탕으로)



In [ ]:
user_type = users.copy()

In [ ]:
vod_log_89 = vod_log[vod_log['month'].isin([8,9])]

In [ ]:
# 사용자별 상세보기 vod수
user_type = user_type.merge(cont_log.groupby(['subsr_id', 'program_id']).count()[['program_name']].groupby('subsr_id').count().reset_index().rename(columns={'program_name':'상세보기vod수'}), how='left')

# 사용자별 시청 vod수
user_type = user_type.merge(vod_log_89.groupby(['subsr_id', 'program_id']).count()[['program_name']].groupby('subsr_id').count().reset_index().rename(columns={'program_name':'시청vod수'}), how='left')

# 사용자별 방문빈도수 (day기반)
user_type = user_type.merge(cont_log.groupby(['subsr_id','month', 'day']).count()[['program_id']].groupby(['subsr_id', 'month']).count().groupby('subsr_id').sum([['program_id']]).reset_index().rename(columns={'program_id':'방문빈도수'}), how='left')

# 사용자별 상세보기조회빈도(클릭수)
user_type = user_type.merge(cont_log.groupby(['subsr_id']).count()[['program_id']].reset_index().rename(columns={'program_id':'상세보기조회빈도(클릭수)'}), how='left')

# 사용자별 평균 클릭수 (클릭수 / 방문빈도수)
user_type['평균 클릭수'] = user_type['상세보기조회빈도(클릭수)'] / user_type['방문빈도수']

# 사용자별 시청빈도(시청수)
user_type = user_type.merge(vod_log_89.groupby(['subsr_id']).count()[['program_id']].reset_index().rename(columns={'program_id':'시청빈도(시청수)'}), how='left')

# 사용자별 평균 시청수 (시청수 / 방문빈도수)
user_type['평균 시청수'] = user_type['시청빈도(시청수)'] / user_type['방문빈도수']

# 사용자별 시청시간합계 (use_ts)
user_type = user_type.merge(vod_log_89.groupby(['subsr_id']).sum()[['use_tms']].reset_index().rename(columns={'use_tms':'시청시간합계'}), how='left')

# 사용자별 평균 시청시간 (시청시가합계 / 방문빈도수)
user_type['평균 시청시간'] = user_type['시청시간합계'] / user_type['방문빈도수']

# 사용자별 vod 시청 비율 (완전한 시청에 대한 기준?)
vod_log_89['use_tms_ratio'] = vod_log_89['use_tms'] / vod_log_89['disp_rtm_sec']
user_type = user_type.merge(vod_log_89.groupby(['subsr_id']).mean()[['use_tms_ratio']].reset_index().rename(columns={'use_tms_ratio':'vod평균시청율'}), how='left')

# 시청전환율
user_type['시청전환율'] = user_type['시청vod수'] / user_type['상세보기vod수']

# age_limit (청불 시청 여부) - 근데 이건 부정확할 수 있음
# 근데, 둘다 1이라면, 분리가 필요하다고 말할 수 있을 듯!
user_type.loc[user_type[user_type['subsr_id'].isin(cont_log[cont_log['age_limit']=='청불']['subsr_id'].unique())].index, '청불여부_상세보기'] = 1
user_type['청불여부_상세보기'] = user_type['청불여부_상세보기'].fillna(0)

user_type.loc[user_type[user_type['subsr_id'].isin(vod_log_89[vod_log_89['age_limit']=='청불']['subsr_id'].unique())].index, '청불여부_시청'] = 1
user_type['청불여부_시청'] = user_type['청불여부_시청'].fillna(0)

In [ ]:
#user_type

# 1) ColdStart_user: 8,9월 기준 어떤 기록도 없는 유저 == 0
user_type.loc[user_type[(user_type['상세보기vod수'].isnull()) & (user_type['시청vod수'].isnull())].index, 'user_type'] = 0

# 2) Starter_user: 8,9월 기준 시청빈도 25% 이하 == 1
user_type.loc[user_type[(user_type['user_type'].isnull()) & (user_type['시청빈도(시청수)'] <= user_type['시청빈도(시청수)'].quantile(0.4))].index, 'user_type'] = 1

# 4) Heavy_user: 8,9월 기준 시청빈도 75% 이상 (상위25%) == 3
user_type.loc[user_type[(user_type['user_type'].isnull()) & (user_type['시청빈도(시청수)'] >= user_type['시청빈도(시청수)'].quantile(0.6))].index, 'user_type'] = 3

# 3) Standard_user == 2 (나머지이므로 마지막에 남은거 할당!)
user_type.loc[user_type[user_type['user_type'].isnull()].index, 'user_type'] = 2

In [ ]:
sns.heatmap(user_type.iloc[4:].corr(), annot=True)

In [ ]:
user_type['user_type'].value_counts()

2.0    557
1.0    188
3.0    178
0.0     28
Name: user_type, dtype: int64

In [ ]:
user_info = user_type[['subsr', 'subsr_id', 'kids', 'user_type']]

In [ ]:
vod_info[vod_info['program_id']=='361']

,program_id,program_name,ct_cl,program_genre,release_date,age_limit
361,361,러브 포 세일,영화,"드라마, 성인",2016.0,청불


### 필요 함수

In [ ]:
def vod_lookup(value):
    result = vod_info[vod_info['program_id']==value].program_name.values[0]
    return result

In [ ]:
def check_recommend_list(model):
    df = pd.DataFrame()
    for i, user in enumerate(tqdm(user_info['subsr_id'])):
        pred = model.recommend(user, model.score_matrix, model.model, 10)['program_id'].values
        df.loc[i, 'subsr_id'] = user
        for rank in range(len(pred)):
            df.loc[i, f'vod_{rank}'] = pred[rank]

    for i in range(10):
        df[f'vod_{i}'] = df[f'vod_{i}'].apply(vod_lookup)

    return df

In [ ]:
def model_performance(model):
    precision = model.precision
    recall = model.recall
    map = model.map
    mar = model.mar
    test_diversity = model.test_diversity
    all_diversity = model.all_diversity
    print('\nPrecision@10: ', round(model.precision, 5))
    print('Recall@10: ', round(model.recall, 5))
    print('MAP@10: ', round(model.map, 5))
    print('MAR@10: ', round(model.mar, 5))
    print('test_Diversity: ', round(model.test_diversity, 5))
    print('all_Diversity: ', round(model.all_diversity, 5))
    return precision, recall, map, mar, test_diversity, all_diversity

In [ ]:
def user_type_performance(user_metrics, rec_list, user_info):
    # 군집별 precision & recall
    df = user_metrics.merge(user_info[['subsr_id', 'user_type']], how='left')
    type_0_pre = df[df['user_type']==0]['precision'].mean()
    type_0_rec = df[df['user_type']==0]['recall'].mean()
    type_1_pre = df[df['user_type']==1]['precision'].mean()
    type_1_rec = df[df['user_type']==1]['recall'].mean()
    type_2_pre = df[df['user_type']==2]['precision'].mean()
    type_2_rec = df[df['user_type']==2]['recall'].mean()
    type_3_pre = df[df['user_type']==3]['precision'].mean()
    type_3_rec = df[df['user_type']==3]['recall'].mean()

    # 군집별 diversity
    df2 = rec_list.merge(user_info[['subsr_id', 'user_type']], how='left')
    type_0_list = df2[df2['user_type']==0]
    type_0_div = 1 - np.sum(cosine_similarity(type_0_list.iloc[:, 1:-1]) - np.eye(len(type_0_list))) / (len(type_0_list) * (len(type_0_list) - 1))
    type_1_list = df2[df2['user_type']==1]
    type_1_div = 1 - np.sum(cosine_similarity(type_1_list.iloc[:, 1:-1]) - np.eye(len(type_1_list))) / (len(type_1_list) * (len(type_1_list) - 1))
    type_2_list = df2[df2['user_type']==2]
    type_2_div = 1 - np.sum(cosine_similarity(type_2_list.iloc[:, 1:-1]) - np.eye(len(type_2_list))) / (len(type_2_list) * (len(type_2_list) - 1))
    type_3_list = df2[df2['user_type']==3]
    type_3_div = 1 - np.sum(cosine_similarity(type_3_list.iloc[:, 1:-1]) - np.eye(len(type_3_list))) / (len(type_3_list) * (len(type_3_list) - 1))

    return type_0_pre, type_0_rec, type_0_div, type_1_pre, type_1_rec, type_1_div, type_2_pre, type_2_rec, type_2_div, type_3_pre, type_3_rec, type_3_div

In [ ]:
def performance_list(model, user_info):
    precision = model.precision
    recall = model.recall
    map = model.map
    mar = model.mar
    test_diversity = model.test_diversity
    #all_diversity = model.all_diversity

    # 군집별 precision & recall
    df = model.user_metrics.merge(user_info[['subsr_id', 'user_type']], how='left')
    type_0_pre = df[df['user_type']==0]['precision'].mean()
    type_0_rec = df[df['user_type']==0]['recall'].mean()
    type_1_pre = df[df['user_type']==1]['precision'].mean()
    type_1_rec = df[df['user_type']==1]['recall'].mean()
    type_2_pre = df[df['user_type']==2]['precision'].mean()
    type_2_rec = df[df['user_type']==2]['recall'].mean()
    type_3_pre = df[df['user_type']==3]['precision'].mean()
    type_3_rec = df[df['user_type']==3]['recall'].mean()

    # 군집별 diversity
    df2 = model.result_list.merge(user_info[['subsr_id', 'user_type']], how='left')
    type_0_list = df2[df2['user_type']==0]
    type_0_div = 1 - np.sum(cosine_similarity(type_0_list.iloc[:, 1:-1]) - np.eye(len(type_0_list))) / (len(type_0_list) * (len(type_0_list) - 1))
    type_1_list = df2[df2['user_type']==1]
    type_1_div = 1 - np.sum(cosine_similarity(type_1_list.iloc[:, 1:-1]) - np.eye(len(type_1_list))) / (len(type_1_list) * (len(type_1_list) - 1))
    type_2_list = df2[df2['user_type']==2]
    type_2_div = 1 - np.sum(cosine_similarity(type_2_list.iloc[:, 1:-1]) - np.eye(len(type_2_list))) / (len(type_2_list) * (len(type_2_list) - 1))
    type_3_list = df2[df2['user_type']==3]
    type_3_div = 1 - np.sum(cosine_similarity(type_3_list.iloc[:, 1:-1]) - np.eye(len(type_3_list))) / (len(type_3_list) * (len(type_3_list) - 1))

    return precision, recall, map, mar, test_diversity, type_0_pre, type_0_rec, type_0_div, type_1_pre, type_1_rec, type_1_div, type_2_pre, type_2_rec, type_2_div, type_3_pre, type_3_rec, type_3_div

### lightfm_ver1

In [ ]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808327 sha256=83565209e42c7cb09acf257cbd3a18bfffc933fdeb1494df324e0282aaea15be
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from lightfm import LightFM, cross_validation
from lightfm.data import Dataset

from scipy.sparse import coo_matrix

In [ ]:
class LightFM_Model:
    def __init__(self, vod_log, cont_log, vod_info, user_info, rating_type, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05):
        self.vod_log = vod_log
        self.cont_log = cont_log
        self.rating_type = rating_type
        self.vod_info = vod_info.astype({'program_id':int})
        self.user_info = user_info
        self.vod_log89, self.test = self.split_month(self.vod_log)
        self.train = self.create_rating_df(self.vod_log89, self.cont_log)
        self.data = self.create_rating_df(self.vod_log, self.cont_log)

        # self.train, self.test = self.split_evaluate(data)
        self.train_interactions, self.train_weights = self.dataset(self.train)

        self.score_matrix = self.create_score_matrix(self.data)
        self.score_matrix_evaluate = self.create_score_matrix(self.train)
        # ####
        self.loss = param_loss
        self.components = param_components
        self.epoch = param_epochs
        self.learning_schedule = param_learning_schedule
        self.learning_rate = param_learning_rate
        self.evaluate_model = self.fit(self.train_interactions, self.train_weights)
        self.precision, self.recall, self.map, self.mar, self.test_diversity, self.user_metrics, self.result_list = self.evaluate(self.train_interactions, self.train_weights, self.test)
        self.model = self.train_model(self.data)
       # self.all_diversity = self.evaluate_all(self.model)

    def split_month(self, vod_log):
        vod_log89 = vod_log[vod_log['month'].isin([8, 9])]
        vod_log10 = vod_log[vod_log['month'].isin([10])]
        vod_log89['program_id'] = vod_log89['program_id'].astype('int')
        vod_log10['program_id'] = vod_log10['program_id'].astype('int')
        return vod_log89, vod_log10

    def create_rating_df(self, vod_log, cont_log):
        vod_log['use_tms_ratio'] = vod_log['use_tms'] / vod_log['disp_rtm_sec']
        vod_log['log_dt'] = pd.to_datetime(vod_log['log_dt'])
        cont_log['log_dt'] = pd.to_datetime(cont_log['log_dt'])
        cont_log['recency'] = (cont_log['log_dt'].max() - cont_log['log_dt']).dt.days # 최근성

        log = pd.concat([vod_log[['subsr_id', 'program_id']], cont_log[['subsr_id', 'program_id']]]).drop_duplicates().reset_index(drop=True)
        log = log.merge(cont_log.groupby(['subsr_id', 'program_id'])[['program_name']].count().reset_index().rename(columns={'program_name':'click_cnt'}), how='left')
        log = log.merge(cont_log.groupby(['subsr_id', 'program_id'])[['recency']].min().reset_index().rename(columns={'recency':'click_recency'}), how='left')

        # (subsr_id, program_id) 쌍에 대해 하나의 평점만 남겨야 함
        # => use_tms_ratio 는 최대값으로 남기고, 시청 count도 해서 추가한다.
        log = log.merge(vod_log.groupby(['subsr_id', 'program_id'])[['use_tms_ratio']].max().reset_index().rename(columns={'use_tms_ratio':'watch_tms_max'}), how='left')
        log = log.merge(vod_log.groupby(['subsr_id', 'program_id'])[['use_tms_ratio']].count().reset_index().rename(columns={'use_tms_ratio':'watch_cnt'}), how='left')

        replace_value = log['click_cnt'].quantile(0.95) # 95% 로 최대값 고정
        log.loc[log[log['click_cnt'] > replace_value].index, 'click_cnt'] = replace_value

        log['click_recency'] = log['click_recency'].max() - log['click_recency']
        replace_value = log['watch_cnt'].quantile(0.95)  # 95% 로 최대값 고정
        log.loc[log[log['watch_cnt'] > replace_value].index, 'watch_cnt'] = replace_value

        if self.rating_type == 1:
            # 시청기록만 사용
            log_rating = log.copy()
            log_rating['rating'] = log_rating['watch_tms_max']
            log_rating.loc[log_rating[log_rating['rating']>0].index, '시청여부'] = 1

        elif self.rating_type == 2:
            log_rating = log.copy()
            log_rating.loc[log_rating[log_rating['watch_cnt'] > 0].index, '시청여부'] = 1
            scaler = MinMaxScaler()
            log_rating.iloc[:, 2:-1] = pd.DataFrame(scaler.fit_transform(log_rating.iloc[:, 2:-1]), columns=log_rating.columns[2:-1]).fillna(0)
            log_rating['rating'] = log_rating['click_cnt'].fillna(0) + log_rating['click_recency'].fillna(0) + log_rating['watch_tms_max'].fillna(0) + log_rating['watch_cnt'].fillna(0)

        elif self.rating_type == 3:
            log_rating = log.copy()
            log_rating.loc[log_rating[log_rating['watch_cnt'] > 0].index, '시청여부'] = 1
            scaler = MinMaxScaler()
            log_rating.iloc[:, 2:-1] = pd.DataFrame(scaler.fit_transform(log_rating.iloc[:, 2:-1]), columns=log_rating.columns[2:-1])
            weight = 0.3
            log_rating['rating'] = (log_rating['click_cnt'].fillna(0) + weight) * (log_rating['click_recency'].fillna(0) + weight) * (log_rating['watch_tms_max'].fillna(0) + weight) * (log_rating['watch_cnt'].fillna(0) + weight)

        elif self.rating_type == 4:
            log_rating = log.copy()
            log_rating.loc[log_rating[log_rating['watch_cnt'] > 0].index, '시청여부'] = 1

            scaler = MinMaxScaler()
            log_rating.iloc[:, 2:-1] = pd.DataFrame(scaler.fit_transform(log_rating.iloc[:, 2:-1]), columns=log_rating.columns[2:-1]).fillna(0)
            pca = PCA(n_components=1, svd_solver='auto') # 전체 feature 수 입력
            pca_result = pca.fit_transform(log_rating.iloc[:, 2:-1])
            log_rating['rating'] = pca_result
            log_rating['rating'] = scaler.fit_transform(log_rating[['rating']])

        rating_df = log_rating[['subsr_id', 'program_id', 'rating', '시청여부']]
        rating_df['program_id'] = rating_df['program_id'].astype('int')
        return rating_df

    def create_score_matrix(self, data):
        df = data.copy()
        df.loc[df[(df['시청여부']==1) & df['rating'].notnull()].index, 'score_matrix'] = 1
        # score_matrix = df.pivot(columns='program_id', index='subsr_id', values='score_matrix')
        score_matrix = pd.DataFrame(np.full((self.data['subsr_id'].max() + 1, self.data['program_id'].max() + 1), np.nan))
        for index, row in df[df['시청여부'].notnull()].iterrows():
            subsr_id = int(row['subsr_id'])
            program_id = int(row['program_id'])
            score = row['시청여부']
            # 빈 행렬에 값 채우기
            score_matrix.loc[subsr_id, program_id] = score
        return score_matrix

    def dataset(self, train):
        dataset = Dataset()
        dataset.fit(users = self.data['subsr_id'].sort_values().unique(),
                    items = self.data['program_id'].sort_values().unique())
        num_users, num_vods = dataset.interactions_shape()
        ####
        train['interaction'] = train['rating'].apply(lambda x: 0 if np.isnan(x) else 1)
        train_dropna = train.dropna()
        train_interactions = coo_matrix((train_dropna['interaction'], (train_dropna['subsr_id'], train_dropna['program_id'])), shape=(self.data['subsr_id'].max() + 1, self.data['program_id'].max() +1))
        train_weights = coo_matrix((train_dropna['rating'], (train_dropna['subsr_id'], train_dropna['program_id'])), shape=(self.data['subsr_id'].max() + 1, self.data['program_id'].max() +1))
        #(train_interactions, train_weights) = dataset.build_interactions(train[['subsr_id', 'program_id', 'rating']].dropna().values)
        return train_interactions, train_weights

    def fit(self, fitting_interactions, fitting_weights):
        model = LightFM(random_state=0, loss=self.loss, no_components=self.components, learning_schedule=self.learning_schedule, learning_rate=self.learning_rate)
        model.fit(interactions=fitting_interactions, sample_weight=fitting_weights, verbose=1, epochs=self.epoch)
        return model

    def predict(self, subsr_id, program_id, model):
        pred = model.predict([subsr_id], [program_id])
        return pred

    def recommend(self, subsr_id, score_matrix, model, N):
        # 안 본 vod 추출
        user_rated = score_matrix.loc[subsr_id].dropna().index.tolist()
        user_unrated = score_matrix.columns.drop(user_rated).tolist()
        # 안본 vod에 대해서 예측하기
        predictions = model.predict(int(subsr_id), user_unrated)
        result = pd.DataFrame({'program_id':user_unrated, 'pred_rating':predictions})
        # pred값에 따른 정렬해서 결과 띄우기
        top_N = result.sort_values(by='pred_rating', ascending=False)[:N]
        return top_N

    @staticmethod
    def precision_recall_at_k(target, prediction):
        num_hit = len(set(prediction).intersection(set(target)))
        precision = float(num_hit) / len(prediction) if len(prediction) > 0 else 0.0
        recall = float(num_hit) / len(target) if len(target) > 0 else 0.0
        return precision, recall

    @staticmethod
    def map_at_k(target, prediction, k=10):
        num_hits = 0
        precision_at_k = 0.0
        for i, p in enumerate(prediction[:k]):
            if p in target:
                num_hits += 1
                precision_at_k += num_hits / (i + 1)
        if not target.any():
            return 0.0
        return precision_at_k / min(k, len(target))

    @staticmethod
    def mar_at_k(target, prediction, k=10):
        num_hits = 0
        recall_at_k = 0.0
        for i, p in enumerate(prediction[:k]):
            if p in target:
                num_hits += 1
                recall_at_k += num_hits / len(target)
        if not target.any():
            return 0.0
        return recall_at_k / min(k, len(target))

    def evaluate(self, train_interactions, train_weights, test, N=10):
        evaluate_model = self.fit(train_interactions, train_weights)
        result = pd.DataFrame()
        precisions = []
        recalls = []
        map_values = []
        mar_values = []
        user_metrics = []

        for idx, user in enumerate(tqdm(test['subsr_id'].unique())):
            targets = test[test['subsr_id']==user]['program_id'].unique()
            predictions = self.recommend(user, self.score_matrix_evaluate, evaluate_model, N)['program_id'].values
            precision, recall = self.precision_recall_at_k(targets, predictions)
            map_at_k_value = self.map_at_k(targets, predictions)
            mar_at_k_value = self.mar_at_k(targets, predictions)
            precisions.append(precision)
            recalls.append(recall)
            map_values.append(map_at_k_value)
            mar_values.append(mar_at_k_value)
            user_metrics.append({'subsr_id': user, 'precision': precision, 'recall': recall, 'map_at_k': map_at_k_value, 'mar_at_k': mar_at_k_value})

            result.loc[idx, 'subsr_id'] = user

            for rank in range(len(predictions)):
                result.loc[idx, f'vod_{rank}'] = predictions[rank]

        list_sim = cosine_similarity(result.iloc[:, 1:])
        list_similarity = np.sum(list_sim - np.eye(len(result))) / (len(result) * (len(result) - 1))
        #print(result)
        # def vod_lookup(value):
        #     name = self.vod_info[self.vod_info['program_id']==value].program_name.values[0]
        #     return name

        # for i in range(len(predictions)):
        #     #print(i)
        #     result[f'vod_{i}'] = result[f'vod_{i}'].apply(vod_lookup)

        return np.mean(precisions), np.mean(recalls), np.mean(map_values), np.mean(mar_values), 1-list_similarity, pd.DataFrame(user_metrics), result


    def evaluate_all(self, model, N=10):
        result = pd.DataFrame()
        for idx, user in enumerate(tqdm(self.user_info['subsr_id'])):
            predictions = self.recommend(user, self.score_matrix, model, N)['program_id'].values
            result.loc[idx, 'subsr_id'] = user
            for rank in range(len(predictions)):
                result.loc[idx, f'vod_{rank}'] = predictions[rank]

        list_sim = cosine_similarity(result.iloc[:, 1:])
        list_similarity = np.sum(list_sim - np.eye(len(result))) / (len(result) * (len(result) - 1))
        return 1 - list_similarity

    def train_model(self, data):
        # 최종 학습 데이터셋 만들기
        train_interactions, train_weights = self.dataset(data)
        model = self.fit(train_interactions, train_weights)
        return model

#### LightFM_ver1 적용 예시

In [ ]:
a = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=2)

100%|██████████| 951/951 [00:03<00:00, 242.72it/s]


In [ ]:
a = LightFM_Model(vod_log, cont_log, vod_info, user_info, 'warp', 30, 10)
print('\nPrecision@10: ', round(a.precision, 5))
print('Recall@10: ', round(a.recall, 5))
print('MAP@10: ', round(a.map, 5))
print('MAR@10: ', round(a.mar, 5))
print('test_Diversity: ', round(a.test_diversity, 5))
print('all_Diversity: ', round(a.all_diversity, 5))

100%|██████████| 951/951 [00:08<00:00, 117.64it/s]



Precision@10:  0.01868
Recall@10:  0.07239
MAP@10:  0.02992
MAR@10:  0.05233
test_Diversity:  0.45088
all_Diversity:  0.41966


### lightfm_ver2

In [ ]:
# 수정된 부분 고쳐서 돌아가는거 확인하기
class LightFM_Model_ver2:
    def __init__(self, vod_log, cont_log, vod_info, user_info, rating_type, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05):
        self.vod_log = vod_log
        self.cont_log = cont_log
        self.rating_type = rating_type
        self.vod_info = vod_info.astype({'program_id':int})
        self.user_info = user_info
        self.vod_log89, self.test = self.split_month(self.vod_log)
        self.train = self.create_rating_df(self.vod_log89, self.cont_log)
        self.data = self.create_rating_df(self.vod_log, self.cont_log)

        ###### 여기부터 고쳐야 할듯
        self.train_interactions, self.train_weights, self.item_features_dataset, self.user_features_dataset, self.item_features_col, self.user_features_col= self.dataset(self.train)
        self.score_matrix = self.create_score_matrix(self.data)
        self.score_matrix_evaluate = self.create_score_matrix(self.train)
        # ####
        self.loss = param_loss
        self.components = param_components
        self.epoch = param_epochs
        self.learning_schedule = param_learning_schedule
        self.learning_rate = param_learning_rate
        self.evaluate_model = self.fit(self.train_interactions, self.train_weights, self.item_features_dataset, self.user_features_dataset)
        self.precision, self.recall, self.map, self.mar, self.test_diversity, self.user_metrics, self.result_list = self.evaluate(self.train_interactions, self.train_weights, self.test)
        self.model = self.train_model(self.data)
        # self.all_diversity = self.evaluate_all(self.model)

    def split_month(self, vod_log):
        vod_log89 = vod_log[vod_log['month'].isin([8, 9])]
        vod_log10 = vod_log[vod_log['month'].isin([10])]
        vod_log89['program_id'] = vod_log89['program_id'].astype('int')
        vod_log10['program_id'] = vod_log10['program_id'].astype('int')
        return vod_log89, vod_log10

    def create_rating_df(self, vod_log, cont_log):
        vod_log['use_tms_ratio'] = vod_log['use_tms'] / vod_log['disp_rtm_sec']
        vod_log['log_dt'] = pd.to_datetime(vod_log['log_dt'])
        cont_log['log_dt'] = pd.to_datetime(cont_log['log_dt'])
        cont_log['recency'] = (cont_log['log_dt'].max() - cont_log['log_dt']).dt.days # 최근성

        log = pd.concat([vod_log[['subsr_id', 'program_id']], cont_log[['subsr_id', 'program_id']]]).drop_duplicates().reset_index(drop=True)
        log = log.merge(cont_log.groupby(['subsr_id', 'program_id'])[['program_name']].count().reset_index().rename(columns={'program_name':'click_cnt'}), how='left')
        log = log.merge(cont_log.groupby(['subsr_id', 'program_id'])[['recency']].min().reset_index().rename(columns={'recency':'click_recency'}), how='left')

        # (subsr_id, program_id) 쌍에 대해 하나의 평점만 남겨야 함
        # => use_tms_ratio 는 최대값으로 남기고, 시청 count도 해서 추가한다.
        log = log.merge(vod_log.groupby(['subsr_id', 'program_id'])[['use_tms_ratio']].max().reset_index().rename(columns={'use_tms_ratio':'watch_tms_max'}), how='left')
        log = log.merge(vod_log.groupby(['subsr_id', 'program_id'])[['use_tms_ratio']].count().reset_index().rename(columns={'use_tms_ratio':'watch_cnt'}), how='left')

        replace_value = log['click_cnt'].quantile(0.95) # 95% 로 최대값 고정
        log.loc[log[log['click_cnt'] > replace_value].index, 'click_cnt'] = replace_value

        log['click_recency'] = log['click_recency'].max() - log['click_recency']
        replace_value = log['watch_cnt'].quantile(0.95)  # 95% 로 최대값 고정
        log.loc[log[log['watch_cnt'] > replace_value].index, 'watch_cnt'] = replace_value

        if self.rating_type == 1:
            # 시청기록만 사용
            log_rating = log.copy()
            log_rating['rating'] = log_rating['watch_tms_max']
            log_rating.loc[log_rating[log_rating['rating']>0].index, '시청여부'] = 1

        elif self.rating_type == 2:
            log_rating = log.copy()
            log_rating.loc[log_rating[log_rating['watch_cnt'] > 0].index, '시청여부'] = 1
            scaler = MinMaxScaler()
            log_rating.iloc[:, 2:-1] = pd.DataFrame(scaler.fit_transform(log_rating.iloc[:, 2:-1]), columns=log_rating.columns[2:-1]).fillna(0)
            log_rating['rating'] = log_rating['click_cnt'].fillna(0) + log_rating['click_recency'].fillna(0) + log_rating['watch_tms_max'].fillna(0) + log_rating['watch_cnt'].fillna(0)

        elif self.rating_type == 3:
            log_rating = log.copy()
            log_rating.loc[log_rating[log_rating['watch_cnt'] > 0].index, '시청여부'] = 1
            scaler = MinMaxScaler()
            log_rating.iloc[:, 2:-1] = pd.DataFrame(scaler.fit_transform(log_rating.iloc[:, 2:-1]), columns=log_rating.columns[2:-1])
            weight = 0.3
            log_rating['rating'] = (log_rating['click_cnt'].fillna(0) + weight) * (log_rating['click_recency'].fillna(0) + weight) * (log_rating['watch_tms_max'].fillna(0) + weight) * (log_rating['watch_cnt'].fillna(0) + weight)

        elif self.rating_type == 4:
            log_rating = log.copy()
            log_rating.loc[log_rating[log_rating['watch_cnt'] > 0].index, '시청여부'] = 1

            scaler = MinMaxScaler()
            log_rating.iloc[:, 2:-1] = pd.DataFrame(scaler.fit_transform(log_rating.iloc[:, 2:-1]), columns=log_rating.columns[2:-1]).fillna(0)
            pca = PCA(n_components=1, svd_solver='auto') # 전체 feature 수 입력
            pca_result = pca.fit_transform(log_rating.iloc[:, 2:-1])
            log_rating['rating'] = pca_result
            log_rating['rating'] = scaler.fit_transform(log_rating[['rating']])

        rating_df = log_rating[['subsr_id', 'program_id', 'rating', '시청여부']]
        rating_df['program_id'] = rating_df['program_id'].astype('int')
        return rating_df

    def create_score_matrix(self, data):
        df = data.copy()
        df.loc[df[(df['시청여부']==1) & df['rating'].notnull()].index, 'score_matrix'] = 1
        # score_matrix = df.pivot(columns='program_id', index='subsr_id', values='score_matrix')
        score_matrix = pd.DataFrame(np.full((self.data['subsr_id'].max() + 1, self.data['program_id'].max() + 1), np.nan))
        for index, row in df[df['시청여부'].notnull()].iterrows():
            subsr_id = int(row['subsr_id'])
            program_id = int(row['program_id'])
            score = row['시청여부']
            # 빈 행렬에 값 채우기
            score_matrix.loc[subsr_id, program_id] = score
        return score_matrix

    def dataset(self, train):
        # item feature 준비
        item_features = self.vod_info[['program_id', 'program_genre']]
        item_features['program_genre'] = [x.split(', ') for x in item_features['program_genre']]
        item_features = pd.concat([item_features.iloc[:, 0], item_features['program_genre'].str.join('|').str.get_dummies()], axis=1)
        item_features_col = item_features.drop(columns=['program_id']).columns.values
        item_feat = item_features.drop(columns=['program_id']).to_dict(orient='records')

        # user feature 준비
        user_features = self.user_info
        user_features_col = user_features.drop('subsr_id', axis=1).columns.values
        user_feat = user_features.drop(columns=['subsr_id']).to_dict(orient='records')

        # dataset 생성
        dataset = Dataset()
        dataset.fit(users = self.data['subsr_id'].sort_values().unique(),
                    items = self.data['program_id'].sort_values().unique(),
                    item_features = item_features_col,
                    user_features = user_features_col)
        #num_users, num_vods = dataset.interactions_shape()
        ####
        train['interaction'] = train['rating'].apply(lambda x: 0 if np.isnan(x) else 1)
        train_dropna = train.dropna()
        train_interactions = coo_matrix((train_dropna['interaction'], (train_dropna['subsr_id'], train_dropna['program_id'])), shape=(self.data['subsr_id'].max() + 1, self.data['program_id'].max() +1))
        train_weights = coo_matrix((train_dropna['rating'], (train_dropna['subsr_id'], train_dropna['program_id'])), shape=(self.data['subsr_id'].max() + 1, self.data['program_id'].max() +1))
        #(train_interactions, train_weights) = dataset.build_interactions(train[['subsr_id', 'program_id', 'rating']].dropna().values)

        item_features_dataset = dataset.build_item_features((x, y) for x, y in zip(item_features['program_id'], item_feat))
        user_features_dataset = dataset.build_user_features((x, y) for x, y in zip(user_features['subsr_id'], user_feat))

        return train_interactions, train_weights, item_features_dataset, user_features_dataset, item_features_col, user_features_col

    def fit(self, fitting_interactions, fitting_weights, item_features_dataset, user_features_dataset):
        model = LightFM(random_state=0, loss=self.loss, no_components=self.components)
        model.fit(interactions=fitting_interactions, sample_weight=fitting_weights
                  ,item_features=item_features_dataset, user_features=user_features_dataset, verbose=1, epochs=self.epoch)
        return model

    def predict(self, subsr_id, program_id, model):
        pred = model.predict([subsr_id], [program_id])
        return pred

    def recommend(self, subsr_id, score_matrix, model, N):
        # 안 본 vod 추출
        user_rated = score_matrix.loc[subsr_id].dropna().index.tolist()
        user_unrated = score_matrix.columns.drop(user_rated).tolist()
        # 안본 vod에 대해서 예측하기
        predictions = model.predict(int(subsr_id), user_unrated)
        result = pd.DataFrame({'program_id':user_unrated, 'pred_rating':predictions})
        # pred값에 따른 정렬해서 결과 띄우기
        top_N = result.sort_values(by='pred_rating', ascending=False)[:N]
        return top_N

    @staticmethod
    def precision_recall_at_k(target, prediction):
        num_hit = len(set(prediction).intersection(set(target)))
        precision = float(num_hit) / len(prediction) if len(prediction) > 0 else 0.0
        recall = float(num_hit) / len(target) if len(target) > 0 else 0.0
        return precision, recall

    @staticmethod
    def map_at_k(target, prediction, k=10):
        num_hits = 0
        precision_at_k = 0.0
        for i, p in enumerate(prediction[:k]):
            if p in target:
                num_hits += 1
                precision_at_k += num_hits / (i + 1)
        if not target.any():
            return 0.0
        return precision_at_k / min(k, len(target))

    @staticmethod
    def mar_at_k(target, prediction, k=10):
        num_hits = 0
        recall_at_k = 0.0
        for i, p in enumerate(prediction[:k]):
            if p in target:
                num_hits += 1
                recall_at_k += num_hits / len(target)
        if not target.any():
            return 0.0
        return recall_at_k / min(k, len(target))

    def evaluate(self, train_interactions, train_weights, test, N=10):
        evaluate_model = self.fit(train_interactions, train_weights, self.item_features_dataset, self.user_features_dataset)
        result = pd.DataFrame()
        precisions = []
        recalls = []
        map_values = []
        mar_values = []
        user_metrics = []

        for idx, user in enumerate(tqdm(test['subsr_id'].unique())):
            targets = test[test['subsr_id']==user]['program_id'].unique()
            predictions = self.recommend(user, self.score_matrix_evaluate, evaluate_model, N)['program_id'].values
            precision, recall = self.precision_recall_at_k(targets, predictions)
            map_at_k_value = self.map_at_k(targets, predictions)
            mar_at_k_value = self.mar_at_k(targets, predictions)
            precisions.append(precision)
            recalls.append(recall)
            map_values.append(map_at_k_value)
            mar_values.append(mar_at_k_value)
            user_metrics.append({'subsr_id': user, 'precision': precision, 'recall': recall, 'map_at_k': map_at_k_value, 'mar_at_k': mar_at_k_value})

            result.loc[idx, 'subsr_id'] = user
            for rank in range(len(predictions)):
                result.loc[idx, f'vod_{rank}'] = predictions[rank]

        list_sim = cosine_similarity(result.iloc[:, 1:])
        list_similarity = np.sum(list_sim - np.eye(len(result))) / (len(result) * (len(result) - 1))

        # def vod_lookup(value):
        #     name = vod_info[vod_info['program_id']==value].program_name.values[0]
        #     return name

        # for i in range(len(predictions)):
        #     result[f'vod_{i}'] = result[f'vod_{i}'].apply(vod_lookup)

        return np.mean(precisions), np.mean(recalls), np.mean(map_values), np.mean(mar_values), 1-list_similarity, pd.DataFrame(user_metrics), result


    def evaluate_all(self, model, N=10):
        result = pd.DataFrame()
        for idx, user in enumerate(tqdm(self.user_info['subsr_id'])):
            predictions = self.recommend(user, self.score_matrix, model, N)['program_id'].values
            result.loc[idx, 'subsr_id'] = user
            for rank in range(len(predictions)):
                result.loc[idx, f'vod_{rank}'] = predictions[rank]

        list_sim = cosine_similarity(result.iloc[:, 1:])
        list_similarity = np.sum(list_sim - np.eye(len(result))) / (len(result) * (len(result) - 1))
        return 1 - list_similarity

    def train_model(self, data):
        # 최종 학습 데이터셋 만들기
        train_interactions, train_weights, item_features_dataset, user_features_dataset, item_features_col, user_features_col = self.dataset(data)
        model = self.fit(train_interactions, train_weights, item_features_dataset, user_features_dataset)
        return model

#### LightFM_ver2 적용 예시

In [ ]:
a = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info, 'warp', 30, 10)
print('\nPrecision@10: ', round(a.precision, 5))
print('Recall@10: ', round(a.recall, 5))
print('MAP@10: ', round(a.map, 5))
print('MAR@10: ', round(a.mar, 5))
print('test_Diversity: ', round(a.test_diversity, 5))
print('all_Diversity: ', round(a.all_diversity, 5))

100%|██████████| 951/951 [00:06<00:00, 150.26it/s]


Precision@10:  0.01905
Recall@10:  0.07628
MAP@10:  0.02351
MAR@10:  0.05647
test_Diversity:  0.57321
all_Diversity:  0.47061


In [ ]:
a = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info, 'warp', 100, 20)
print('\nPrecision@10: ', round(a.precision, 5))
print('Recall@10: ', round(a.recall, 5))
print('MAP@10: ', round(a.map, 5))
print('MAR@10: ', round(a.mar, 5))
print('test_Diversity: ', round(a.test_diversity, 5))
print('all_Diversity: ', round(a.all_diversity, 5))

100%|██████████| 951/951 [00:08<00:00, 107.97it/s]



Precision@10:  0.01722
Recall@10:  0.05929
MAP@10:  0.0229
MAR@10:  0.03921
test_Diversity:  0.57752
all_Diversity:  0.48488


In [ ]:
a = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info, 'bpr', 100, 20)
print('\nPrecision@10: ', round(a.precision, 5))
print('Recall@10: ', round(a.recall, 5))
print('MAP@10: ', round(a.map, 5))
print('MAR@10: ', round(a.mar, 5))
print('test_Diversity: ', round(a.test_diversity, 5))
print('all_Diversity: ', round(a.all_diversity, 5))

100%|██████████| 951/951 [00:13<00:00, 69.57it/s] 


Precision@10:  0.01465
Recall@10:  0.06239
MAP@10:  0.02385
MAR@10:  0.04897
test_Diversity:  0.42603
all_Diversity:  0.40381


### SVD

**[ 파라미터 조정 ]**
- n_factors: 잠재 요인의 개수 (default:100)
- n_epochs: 에포크 수 (default:20)
- lr_all: 전체 학습률 (default: 0.005)
- reg_all: 전체 정규화 강도 (default: 0.02)


In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163744 sha256=1a0588c5bb16bec31858c56cf1c38d86fc47dfd50f5941340b7b2f84e4cfc9fb
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import SVD, Reader, Dataset, accuracy
from sklearn.model_selection import train_test_split

In [ ]:
class SVDRecommendationModel:
    def __init__(self, vod_log, cont_log, vod_info, user_info, rating_type, factors=100, epochs=20, lr=0.005, reg=0.02):
        self.vod_log = vod_log
        self.cont_log = cont_log
        self.rating_type = rating_type
        self.vod_info = vod_info.astype({'program_id':int})
        self.user_info = user_info
        self.vod_log89, self.test = self.split_month(self.vod_log)
        self.train = self.create_rating_df(self.vod_log89, self.cont_log)
        self.data = self.create_rating_df(self.vod_log, self.cont_log)

        self.train_data= self.dataset(self.train, self.test) # suprise 라이브러리 데이터셋 형태 변환

        self.score_matrix = self.create_score_matrix(self.data)
        self.score_matrix_evaluate = self.create_score_matrix(self.train)
        #
        self.n_factors = factors
        self.n_epochs = epochs
        self.lr_all = lr
        self.reg_all = reg
        self.evaluate_model = self.fit(self.train_data)
        self.precision, self.recall, self.map, self.mar, self.test_diversity, self.user_metrics, self.result_list = self.evaluate(self.train_data, self.test)
        self.model = self.train_model(self.data)
        # self.all_diversity = self.evaluate_all(self.model)

    def split_month(self, vod_log):
        vod_log89 = vod_log[vod_log['month'].isin([8, 9])]
        vod_log10 = vod_log[vod_log['month'].isin([10])]
        vod_log89['program_id'] = vod_log89['program_id'].astype('int')
        vod_log10['program_id'] = vod_log10['program_id'].astype('int')
        return vod_log89, vod_log10

    def create_rating_df(self, vod_log, cont_log):
        vod_log['use_tms_ratio'] = vod_log['use_tms'] / vod_log['disp_rtm_sec']
        vod_log['log_dt'] = pd.to_datetime(vod_log['log_dt'])
        cont_log['log_dt'] = pd.to_datetime(cont_log['log_dt'])
        cont_log['recency'] = (cont_log['log_dt'].max() - cont_log['log_dt']).dt.days # 최근성

        log = pd.concat([vod_log[['subsr_id', 'program_id']], cont_log[['subsr_id', 'program_id']]]).drop_duplicates().reset_index(drop=True)
        log = log.merge(cont_log.groupby(['subsr_id', 'program_id'])[['program_name']].count().reset_index().rename(columns={'program_name':'click_cnt'}), how='left')
        log = log.merge(cont_log.groupby(['subsr_id', 'program_id'])[['recency']].min().reset_index().rename(columns={'recency':'click_recency'}), how='left')

        # (subsr_id, program_id) 쌍에 대해 하나의 평점만 남겨야 함
        # => use_tms_ratio 는 최대값으로 남기고, 시청 count도 해서 추가한다.
        log = log.merge(vod_log.groupby(['subsr_id', 'program_id'])[['use_tms_ratio']].max().reset_index().rename(columns={'use_tms_ratio':'watch_tms_max'}), how='left')
        log = log.merge(vod_log.groupby(['subsr_id', 'program_id'])[['use_tms_ratio']].count().reset_index().rename(columns={'use_tms_ratio':'watch_cnt'}), how='left')

        replace_value = log['click_cnt'].quantile(0.95) # 95% 로 최대값 고정
        log.loc[log[log['click_cnt'] > replace_value].index, 'click_cnt'] = replace_value

        log['click_recency'] = log['click_recency'].max() - log['click_recency']
        replace_value = log['watch_cnt'].quantile(0.95)  # 95% 로 최대값 고정
        log.loc[log[log['watch_cnt'] > replace_value].index, 'watch_cnt'] = replace_value

        if self.rating_type == 1:
            # 시청기록만 사용
            log_rating = log.copy()
            log_rating['rating'] = log_rating['watch_tms_max']
            log_rating.loc[log_rating[log_rating['rating']>0].index, '시청여부'] = 1

        elif self.rating_type == 2:
            log_rating = log.copy()
            log_rating.loc[log_rating[log_rating['watch_cnt'] > 0].index, '시청여부'] = 1
            scaler = MinMaxScaler()
            log_rating.iloc[:, 2:-1] = pd.DataFrame(scaler.fit_transform(log_rating.iloc[:, 2:-1]), columns=log_rating.columns[2:-1]).fillna(0)
            log_rating['rating'] = log_rating['click_cnt'].fillna(0) + log_rating['click_recency'].fillna(0) + log_rating['watch_tms_max'].fillna(0) + log_rating['watch_cnt'].fillna(0)

        elif self.rating_type == 3:
            log_rating = log.copy()
            log_rating.loc[log_rating[log_rating['watch_cnt'] > 0].index, '시청여부'] = 1
            scaler = MinMaxScaler()
            log_rating.iloc[:, 2:-1] = pd.DataFrame(scaler.fit_transform(log_rating.iloc[:, 2:-1]), columns=log_rating.columns[2:-1])
            weight = 0.3
            log_rating['rating'] = (log_rating['click_cnt'].fillna(0) + weight) * (log_rating['click_recency'].fillna(0) + weight) * (log_rating['watch_tms_max'].fillna(0) + weight) * (log_rating['watch_cnt'].fillna(0) + weight)

        elif self.rating_type == 4:
            log_rating = log.copy()
            log_rating.loc[log_rating[log_rating['watch_cnt'] > 0].index, '시청여부'] = 1

            scaler = MinMaxScaler()
            log_rating.iloc[:, 2:-1] = pd.DataFrame(scaler.fit_transform(log_rating.iloc[:, 2:-1]), columns=log_rating.columns[2:-1]).fillna(0)
            pca = PCA(n_components=1, svd_solver='auto') # 전체 feature 수 입력
            pca_result = pca.fit_transform(log_rating.iloc[:, 2:-1])
            log_rating['rating'] = pca_result
            log_rating['rating'] = scaler.fit_transform(log_rating[['rating']])

        rating_df = log_rating[['subsr_id', 'program_id', 'rating', '시청여부']]
        rating_df['program_id'] = rating_df['program_id'].astype('int')
        return rating_df

    def create_score_matrix(self, data):
        df = data.copy()
        df.loc[df[(df['시청여부']==1) & df['rating'].notnull()].index, 'score_matrix'] = 1
        # score_matrix = df.pivot(columns='program_id', index='subsr_id', values='score_matrix')
        score_matrix = pd.DataFrame(np.full((self.data['subsr_id'].max() + 1, self.data['program_id'].max() + 1), np.nan))
        for index, row in df[df['시청여부'].notnull()].iterrows():
            subsr_id = int(row['subsr_id'])
            program_id = int(row['program_id'])
            score = row['시청여부']
            # 빈 행렬에 값 채우기
            score_matrix.loc[subsr_id, program_id] = score
        return score_matrix

    # 데이터셋 만들기
    def dataset(self, train, test):
        # surprise dataset 생성
        reader = Reader(rating_scale=(0, 1))
        train_data = Dataset.load_from_df(train[['subsr_id', 'program_id', 'rating']].dropna(), reader)
        train_data = train_data.build_full_trainset()
        return train_data

    def fit(self, fitting_data):
        model = SVD(random_state=0, n_factors=self.n_factors, n_epochs=self.n_epochs, lr_all=self.lr_all, reg_all=self.reg_all)
        model.fit(fitting_data)
        return model

    def predict(self, subsr_id, program_id, model):
        return model.predict(subsr_id, program_id).est

    def recommend(self, subsr_id, score_matrix, model, N):
        user_rated = score_matrix.loc[subsr_id].dropna().index.tolist()
        user_unrated = score_matrix.loc[subsr_id].drop(user_rated).index.tolist()
        predictions = [self.predict(subsr_id, program_id, model) for program_id in user_unrated]
        result = pd.DataFrame({'program_id': user_unrated, 'pred_rating': predictions})
        top_N = result.sort_values(by='pred_rating', ascending=False)[:N]
        return top_N

    @staticmethod
    def precision_recall_at_k(target, prediction):
        num_hit = len(set(prediction).intersection(set(target)))
        precision = float(num_hit) / len(prediction) if len(prediction) > 0 else 0.0
        recall = float(num_hit) / len(target) if len(target) > 0 else 0.0
        return precision, recall

    @staticmethod
    def map_at_k(target, prediction, k=10):
        num_hits = 0
        precision_at_k = 0.0
        for i, p in enumerate(prediction[:k]):
            if p in target:
                num_hits += 1
                precision_at_k += num_hits / (i + 1)
        if not target.any():
            return 0.0
        return precision_at_k / min(k, len(target))

    @staticmethod
    def mar_at_k(target, prediction, k=10):
        num_hits = 0
        recall_at_k = 0.0
        for i, p in enumerate(prediction[:k]):
            if p in target:
                num_hits += 1
                recall_at_k += num_hits / len(target)
        if not target.any():
            return 0.0
        return recall_at_k / min(k, len(target))

    def evaluate(self, train_data, test, N=10):
        # fitting,
        #evaluate_model = self.fit(train_data)
        result = pd.DataFrame()
        precisions = []
        recalls = []
        map_values = []
        mar_values = []
        user_metrics = []

        for idx, user in enumerate(tqdm(test['subsr_id'].unique())):
            targets = test[test['subsr_id']==user]['program_id'].unique()
            predictions = self.recommend(user, self.score_matrix_evaluate, self.evaluate_model, N)['program_id'].values
            precision, recall = self.precision_recall_at_k(targets, predictions)
            map_at_k_value = self.map_at_k(targets, predictions)
            mar_at_k_value = self.mar_at_k(targets, predictions)
            precisions.append(precision)
            recalls.append(recall)
            map_values.append(map_at_k_value)
            mar_values.append(mar_at_k_value)
            user_metrics.append({'subsr_id': user, 'precision': precision, 'recall': recall, 'map_at_k': map_at_k_value, 'mar_at_k': mar_at_k_value})

            result.loc[idx, 'subsr_id'] = user
            for rank in range(len(predictions)):
                result.loc[idx, f'vod_{rank}'] = predictions[rank]

        list_sim = cosine_similarity(result.iloc[:, 1:])
        list_similarity = np.sum(list_sim - np.eye(len(result))) / (len(result) * (len(result) - 1))

        # def vod_lookup(value):
        #     name = vod_info[vod_info['program_id']==value].program_name.values[0]
        #     return name

        # for i in range(len(predictions)):
        #     result[f'vod_{i}'] = result[f'vod_{i}'].apply(vod_lookup)

        return np.mean(precisions), np.mean(recalls), np.mean(map_values), np.mean(mar_values), 1-list_similarity, pd.DataFrame(user_metrics), result

    def evaluate_all(self, model, N=10):
        result = pd.DataFrame()
        for idx, user in enumerate(tqdm(self.user_info['subsr_id'])):
            predictions = self.recommend(user, self.score_matrix, model, N)['program_id'].values
            result.loc[idx, 'subsr_id'] = user
            for rank in range(len(predictions)):
                result.loc[idx, f'vod_{rank}'] = predictions[rank]

        list_sim = cosine_similarity(result.iloc[:, 1:])
        list_similarity = np.sum(list_sim - np.eye(len(result))) / (len(result) * (len(result) - 1))
        return 1 - list_similarity


    def train_model(self, data):
        reader = Reader(rating_scale=(0, 1))
        train_data = Dataset.load_from_df(data[['subsr_id', 'program_id', 'rating']].dropna(), reader)
        train_data = train_data.build_full_trainset()
        model = self.fit(train_data)
        return model

#### SVD 적용 예시

In [ ]:
svd = SVDRecommendationModel(vod_log, cont_log, vod_info, user_info, rating_type=1, factors=100, epochs=20, lr=0.005, reg=0.02)

100%|██████████| 951/951 [00:23<00:00, 40.58it/s]


In [ ]:
print('\nPrecision@10: ', round(svd.precision, 5))
print('Recall@10: ', round(svd.recall, 5))
print('MAP@10: ', round(svd.map, 5))
print('MAR@10: ', round(svd.mar, 5))
print('test_Diversity: ', round(svd.test_diversity, 5))
print('all_Diversity: ', round(svd.all_diversity, 5))


Precision@10:  0.00659
Recall@10:  0.04344
MAP@10:  0.02433
MAR@10:  0.03868
test_Diversity:  0.51241
all_Diversity:  0.49134


### 모델별 TEST 진행하기

In [ ]:
def record_model_performance(model_name, model, user_info, performance_df):
    [precision, recall, map, mar, test_diversity
    , type_0_pre, type_0_rec, type_0_div, type_1_pre, type_1_rec, type_1_div
    , type_2_pre, type_2_rec, type_2_div, type_3_pre, type_3_rec, type_3_div] = performance_list(model, user_info)

    new_row = pd.DataFrame({
        'Model':[model_name],
        'Precision@10':[precision],
        'Recall@10':[recall],
        'MAP@10':[map],
        'MAR@10':[mar],
        'Diversity':[test_diversity],
        '콜드스타트유저_precision':[type_0_pre],
        '비기너유저_precision':[type_1_pre],
        '스탠다드유저_precision':[type_2_pre],
        '헤비유저_precision':[type_3_pre],
        '콜드스타트유저_recall':[type_0_rec],
        '비기너유저_recall':[type_1_rec],
        '스탠다드유저_recall':[type_2_rec],
        '헤비유저_recall':[type_3_rec],
        '콜드스타트유저_diversity':[type_0_div],
        '비기너유저_diversity':[type_1_div],
        '스탠다드유저_diversity':[type_2_div],
        '헤비유저_diversity':[type_3_div]
    })
    # 새로운 행을 성능 DataFrame에 추가
    performance_df = pd.concat([performance_df, new_row], ignore_index=True)

    return performance_df

In [ ]:
performance_df = pd.DataFrame()
#record_model_performance('svd모델',svd, user_info, performance_df)

In [ ]:
performance_df = pd.DataFrame()

SVD 입력인자
: vod_log, cont_log, vod_info, user_info, rating_type, factors=100, epochs=20, lr=0.005, reg=0.02

In [ ]:
svd1 = SVDRecommendationModel(vod_log, cont_log, vod_info, user_info, rating_type=1, factors=100, epochs=20, lr=0.005, reg=0.02)
performance_df = record_model_performance('SVD모델_rating1_기본파라미터', svd1, user_info, performance_df)

svd2 = SVDRecommendationModel(vod_log, cont_log, vod_info, user_info, rating_type=2, factors=100, epochs=20, lr=0.005, reg=0.02)
performance_df = record_model_performance('SVD모델_rating2_기본파라미터', svd2, user_info, performance_df)

svd3 = SVDRecommendationModel(vod_log, cont_log, vod_info, user_info, rating_type=3, factors=100, epochs=20, lr=0.005, reg=0.02)
performance_df = record_model_performance('SVD모델_rating3_기본파라미터', svd3, user_info, performance_df)

svd4 = SVDRecommendationModel(vod_log, cont_log, vod_info, user_info, rating_type=4, factors=100, epochs=20, lr=0.005, reg=0.02)
performance_df = record_model_performance('SVD모델_rating4_기본파라미터', svd4, user_info, performance_df)

100%|██████████| 273/273 [00:10<00:00, 26.33it/s]


In [ ]:
svd1 = SVDRecommendationModel(vod_log, cont_log, vod_info, user_info, rating_type=1, factors=50, epochs=20, lr=0.005, reg=0.02)
performance_df = record_model_performance('SVD모델_rating1_factor축소', svd1, user_info, performance_df)

svd2 = SVDRecommendationModel(vod_log, cont_log, vod_info, user_info, rating_type=2, factors=50, epochs=20, lr=0.005, reg=0.02)
performance_df = record_model_performance('SVD모델_rating2_factor축소', svd2, user_info, performance_df)

svd3 = SVDRecommendationModel(vod_log, cont_log, vod_info, user_info, rating_type=3, factors=50, epochs=20, lr=0.005, reg=0.02)
performance_df = record_model_performance('SVD모델_rating3_factor축소', svd3, user_info, performance_df)

svd4 = SVDRecommendationModel(vod_log, cont_log, vod_info, user_info, rating_type=4, factors=50, epochs=20, lr=0.005, reg=0.02)
performance_df = record_model_performance('SVD모델_rating4_factor축소', svd4, user_info, performance_df)

100%|██████████| 273/273 [00:12<00:00, 22.55it/s]


In [ ]:
df = pd.DataFrame(test_df[['subsr', 'subsr_id']])

In [ ]:
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'모델이름'}).sort_values(by='subsr_id'), how='left')

In [ ]:
df

,subsr,subsr_id
16,64385000,16
22,60593000,22
32,64130000,32
55,64984000,55
89,66056000,89
90,66656000,90
101,66875000,101
125,66705000,125
128,60067000,128
170,62248000,170


In [ ]:
svd1 = SVDRecommendationModel(vod_log, cont_log, vod_info, user_info, rating_type=1, factors=100, epochs=20, lr=0.005, reg=0.02)
# performance_df = record_model_performance('SVD모델_rating1_기본파라미터', svd1, user_info, performance_df)
user_metrics = svd1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'SVD모델_rating1_기본파라미터'}).sort_values(by='subsr_id'), how='left')

svd2 = SVDRecommendationModel(vod_log, cont_log, vod_info, user_info, rating_type=2, factors=100, epochs=20, lr=0.005, reg=0.02)
# performance_df = record_model_performance('SVD모델_rating2_기본파라미터', svd2, user_info, performance_df)
user_metrics = svd2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'SVD모델_rating2_기본파라미터'}).sort_values(by='subsr_id'), how='left')

svd3 = SVDRecommendationModel(vod_log, cont_log, vod_info, user_info, rating_type=3, factors=100, epochs=20, lr=0.005, reg=0.02)
# performance_df = record_model_performance('SVD모델_rating3_기본파라미터', svd3, user_info, performance_df)
user_metrics = svd3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'SVD모델_rating3_기본파라미터'}).sort_values(by='subsr_id'), how='left')


svd4 = SVDRecommendationModel(vod_log, cont_log, vod_info, user_info, rating_type=4, factors=100, epochs=20, lr=0.005, reg=0.02)
# performance_df = record_model_performance('SVD모델_rating4_기본파라미터', svd4, user_info, performance_df)
user_metrics = svd4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'SVD모델_rating4_기본파라미터'}).sort_values(by='subsr_id'), how='left')

100%|██████████| 273/273 [00:23<00:00, 11.71it/s]


LightFM Ver1 인자: vod_log, cont_log, vod_info, user_info, rating_type, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05

In [ ]:
user_info = user_type.copy()

In [ ]:
lfm1 = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=1, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver1모델_rating1_기본파라미터', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver1모델_rating1_기본파라미터'}).sort_values(by='subsr_id'), how='left')

lfm1 = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=2, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver1모델_rating2_기본파라미터', lfm2, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver1모델_rating2_기본파라미터'}).sort_values(by='subsr_id'), how='left')

lfm1 = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=3, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver1모델_rating3_기본파라미터', lfm3, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver1모델_rating3_기본파라미터'}).sort_values(by='subsr_id'), how='left')

lfm1 = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=4, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver1모델_rating4_기본파라미터', lfm4, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver1모델_rating4_기본파라미터'}).sort_values(by='subsr_id'), how='left')

Epoch: 100%|██████████| 10/10 [00:00<00:00, 394.62it/s]


In [ ]:
lfm1 = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=1, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver1모델_rating1_component증가', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver1모델_rating1_component증가'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=2, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver1모델_rating2_component증가', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver1모델_rating2_component증가'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=3, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver1모델_rating3_component증가', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver1모델_rating3_component증가'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=4, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver1모델_rating4_component증가', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver1모델_rating4_component증가'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:00<00:00, 186.34it/s]


In [ ]:
lfm1 = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=1, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver1모델_rating1_loss변경', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver1모델_rating1_loss변경'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=2, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver1모델_rating1_loss변경', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver1모델_rating1_loss변경'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=3, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver1모델_rating1_loss변경', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver1모델_rating1_loss변경'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=4, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver1모델_rating1_loss변경', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver1모델_rating1_loss변경'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:00<00:00, 75.24it/s]


In [ ]:
lfm1 = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=1, param_loss='warp', param_components=20, param_epochs=20, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver1모델_rating1_epoch변경', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver1모델_rating1_epoch변경'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=2, param_loss='warp', param_components=20, param_epochs=20, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver1모델_rating2_epoch변경', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver1모델_rating2_epoch변경'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=3, param_loss='warp', param_components=20, param_epochs=20, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver1모델_rating3_epoch변경', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver1모델_rating3_epoch변경'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model(vod_log, cont_log, vod_info, user_info, rating_type=4, param_loss='warp', param_components=20, param_epochs=20, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver1모델_rating4_epoch변경', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver1모델_rating4_epoch변경'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 20/20 [00:00<00:00, 136.31it/s]


In [ ]:
user_info1= user_info[['subsr_id','user_type']]

In [ ]:
lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info1, rating_type=1, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_rating1_기본파라미터', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_rating1_기본파라미터'}).sort_values(by='subsr_id'), how='left')

lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info1, rating_type=2, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_rating2_기본파라미터', lfm2, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_rating2_기본파라미터'}).sort_values(by='subsr_id'), how='left')

lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info1, rating_type=3, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_rating3_기본파라미터', lfm3, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_rating3_기본파라미터'}).sort_values(by='subsr_id'), how='left')

lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info1, rating_type=4, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_rating4_기본파라미터', lfm4, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_rating4_기본파라미터'}).sort_values(by='subsr_id'), how='left')

Epoch: 100%|██████████| 10/10 [00:00<00:00, 28.55it/s]


In [ ]:
lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info1, rating_type=1, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_rating1_component증가', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_rating1_component증가'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info1, rating_type=2, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_rating2_component증가', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_rating2_component증가'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info1, rating_type=3, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_rating3_component증가', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_rating3_component증가'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info1, rating_type=4, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_rating4_component증가', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_rating4_component증가'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:01<00:00,  7.24it/s]


In [ ]:
lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info1, rating_type=1, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_rating1_loss변경', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_rating1_loss변경'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info1, rating_type=2, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_rating2_loss변경', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_rating2_loss변경'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info1, rating_type=3, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_rating3_loss변경', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_rating3_loss변경'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info1, rating_type=4, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_rating4_loss변경', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_rating4_loss변경'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:03<00:00,  2.98it/s]


In [ ]:
lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info1, rating_type=1, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.01)
# performance_df = record_model_performance('lfm_ver2모델_rating1_learning_rate변경', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_rating1_learning_rate변경'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info1, rating_type=2, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.01)
# performance_df = record_model_performance('lfm_ver2모델_rating2_learning_rate변경', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_rating2_learning_rate변경'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info1, rating_type=3, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.01)
# performance_df = record_model_performance('lfm_ver2모델_rating3_learning_rate변경', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_rating3_learning_rate변경'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_info1, rating_type=4, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.01)
# performance_df = record_model_performance('lfm_ver2모델_rating4_learning_rate변경', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_rating4_learning_rate변경'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:01<00:00,  9.70it/s]


User feature 조정
- 기본:user_type
- user_type_1: 0.2 이상의 상관계수 (시청vod수, 방문빈도수, 시청빈도(시청수),평균 시청수, 시청시간합계,vod평균시청율, 시청전환율, user_type)
- user_type_2: 0.3 이상의 상관계수 (시청빈도(시청수), 평균 시청수, 시청시간합계,vod평균시청율, user_type)
- user_type_3: 0.5 이상의 상관계수 (평균 시청수, 시청시간합계)

In [ ]:
# MinMaxScaler 초기화
scaler = MinMaxScaler()
scaled_user = user_type.copy()

# NaN 값을 가진 행은 스케일링에서 제외
# scaled_values = scaler.fit_transform(scaled_df2[numerical_columns].fillna(scaled_df2[numerical_columns].mean()))

# Assign scaled values back to the DataFrame
# scaled_df2.loc[:, numerical_columns] = scaled_values

# 스케일링된 데이터프레임의 처음 몇 행을 표시
# scaled_df2.head()

# 기존 DataFrame과 NaN 값을 포함하는 DataFrame을 행 기준으로 결합
# combined_df = pd.concat([nan_rows_df, scaled_df2], axis=0, ignore_index=True)

# scaling 후, Nan 값 0으로 채우기
# combined_df_filled = combined_df.fillna(0)
# combined_df_filled
scaled_user.iloc[:, 4:] = scaler.fit_transform(scaled_user.iloc[:, 4:])

In [ ]:
scaled_user.corr()[['평균 시청시간']]

,평균 시청시간
subsr,0.001571
subsr_id,-0.272163
kids,-0.116921
ip,NaN
상세보기vod수,0.041103
시청vod수,0.216584
방문빈도수,0.214246
상세보기조회빈도(클릭수),0.088371
평균 클릭수,0.126119
시청빈도(시청수),0.472010


In [ ]:
user_type_1 = scaled_user[['subsr_id','시청vod수', '방문빈도수', '시청빈도(시청수)', '평균 시청수', '시청시간합계', 'vod평균시청율', '시청전환율', 'user_type']]
user_type_2 = scaled_user[['subsr_id','시청빈도(시청수)', '평균 시청수', '시청시간합계', 'vod평균시청율', 'user_type']]
user_type_3 = scaled_user[['subsr_id', '평균 시청수', '시청시간합계']]

In [ ]:
user_type_1 = user_type_1.fillna(0)

In [ ]:
lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_1, rating_type=1, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype1_rating1_기본파라미터', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype1_rating1_기본파라미터'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_1, rating_type=2, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype1_rating2_기본파라미터', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype1_rating2_기본파라미터'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_1, rating_type=3, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype1_rating3_기본파라미터', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype1_rating3_기본파라미터'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_1, rating_type=4, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype1_rating4_기본파라미터', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype1_rating4_기본파라미터'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:01<00:00,  9.71it/s]


In [ ]:
lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_1, rating_type=1, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype1_rating1_component증가', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype1_rating1_component증가'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_1, rating_type=2, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype1_rating2_component증가', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype1_rating2_component증가'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_1, rating_type=3, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype1_rating3_component증가', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype1_rating3_component증가'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_1, rating_type=4, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype1_rating4_component증가', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype1_rating4_component증가'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:01<00:00,  5.89it/s]


In [ ]:
lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_1, rating_type=1, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype1_rating1_loss변경', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype1_rating1_loss변경'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_1, rating_type=2, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype1_rating2_loss변경', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype1_rating2_loss변경'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_1, rating_type=3, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype1_rating3_loss변경', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype1_rating3_loss변경'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_1, rating_type=4, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype1_rating4_loss변경', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype1_rating4_loss변경'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:03<00:00,  2.97it/s]


In [ ]:
lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_1, rating_type=1, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.01)
# performance_df = record_model_performance('lfm_ver2모델_usertype1_rating1_learning_rate변경', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype1_rating1_learning_rate변경'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_1, rating_type=2, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.01)
# performance_df = record_model_performance('lfm_ver2모델_usertype1_rating2_learning_rate변경', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype1_rating2_learning_rate변경'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_1, rating_type=3, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.01)
# performance_df = record_model_performance('lfm_ver2모델_usertype1_rating3_learning_rate변경', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype1_rating3_learning_rate변경'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_1, rating_type=4, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.01)
# performance_df = record_model_performance('lfm_ver2모델_usertype1_rating4_learning_rate변경', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype1_rating4_learning_rate변경'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:03<00:00,  2.98it/s]


In [ ]:
user_type_2 = user_type_2.fillna(0)

In [ ]:
lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_2, rating_type=1, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype2_rating1_기본파라미터', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype2_rating1_기본파라미터'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_2, rating_type=2, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype2_rating2_기본파라미터', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype2_rating2_기본파라미터'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_2, rating_type=3, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype2_rating3_기본파라미터', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype2_rating3_기본파라미터'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_2, rating_type=4, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype2_rating4_기본파라미터', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype2_rating4_기본파라미터'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:00<00:00, 27.99it/s]


In [ ]:
lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_2, rating_type=1, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype2_rating1_component증가', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype2_rating1_component증가'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_2, rating_type=2, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype2_rating2_component증가', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype2_rating2_component증가'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_2, rating_type=3, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype2_rating3_component증가', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype2_rating3_component증가'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_2, rating_type=4, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype2_rating4_component증가', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype2_rating4_component증가'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:00<00:00, 10.28it/s]


In [ ]:
lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_2, rating_type=1, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype2_rating1_loss변경', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype2_rating1_loss변경'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_2, rating_type=2, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype2_rating2_loss변경', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype2_rating2_loss변경'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_2, rating_type=3, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype2_rating3_loss변경', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype2_rating3_loss변경'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_2, rating_type=4, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype2_rating4_loss변경', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype2_rating4_loss변경'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:01<00:00,  9.49it/s]


In [ ]:
lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_2, rating_type=1, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.01)
# performance_df = record_model_performance('lfm_ver2모델_usertype2_rating1_learning_rate변경', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype2_rating1_learning_rate변경'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_2, rating_type=2, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.01)
# performance_df = record_model_performance('lfm_ver2모델_usertype2_rating2_learning_rate변경', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype2_rating2_learning_rate변경'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_2, rating_type=3, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.01)
# performance_df = record_model_performance('lfm_ver2모델_usertype2_rating3_learning_rate변경', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype2_rating3_learning_rate변경'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_2, rating_type=4, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.01)
# performance_df = record_model_performance('lfm_ver2모델_usertype2_rating4_learning_rate변경', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype2_rating4_learning_rate변경'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:03<00:00,  3.17it/s]


In [ ]:
user_type_3 = user_type_3.fillna(0)

In [ ]:
lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=1, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype3_rating1_기본파라미터', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype3_rating1_기본파라미터'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=2, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype3_rating2_기본파라미터', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype3_rating2_기본파라미터'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=3, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype3_rating3_기본파라미터', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype3_rating3_기본파라미터'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=4, param_loss='logistic', param_components=10, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype3_rating4_기본파라미터', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype3_rating4_기본파라미터'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:00<00:00, 17.08it/s]


In [ ]:
lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=1, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype3_rating1_component증가', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype3_rating1_component증가'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=2, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype3_rating2_component증가', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype3_rating2_component증가'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=3, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype3_rating3_component증가', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype3_rating3_component증가'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=4, param_loss='logistic', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype3_rating4_component증가', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype3_rating4_component증가'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:01<00:00,  7.48it/s]


In [ ]:
lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=1, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype3_rating1_loss변경', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype3_rating1_loss변경'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=2, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype3_rating2_loss변경', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype3_rating2_loss변경'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=3, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype3_rating3_loss변경', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype3_rating3_loss변경'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=4, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)
# performance_df = record_model_performance('lfm_ver2모델_usertype3_rating4_loss변경', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype3_rating4_loss변경'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:02<00:00,  3.95it/s]


In [ ]:
lfm1 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=1, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.01)
# performance_df = record_model_performance('lfm_ver2모델_usertype3_rating1_learning_rate변경', lfm1, user_info, performance_df)
user_metrics = lfm1.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype3_rating1_learning_rate변경'}).sort_values(by='subsr_id'), how='left')

lfm2 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=2, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.01)
# performance_df = record_model_performance('lfm_ver2모델_usertype3_rating2_learning_rate변경', lfm2, user_info, performance_df)
user_metrics = lfm2.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype3_rating2_learning_rate변경'}).sort_values(by='subsr_id'), how='left')

lfm3 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=3, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.01)
# performance_df = record_model_performance('lfm_ver2모델_usertype3_rating3_learning_rate변경', lfm3, user_info, performance_df)
user_metrics = lfm3.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype3_rating3_learning_rate변경'}).sort_values(by='subsr_id'), how='left')

lfm4 = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=4, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.01)
# performance_df = record_model_performance('lfm_ver2모델_usertype3_rating4_learning_rate변경', lfm4, user_info, performance_df)
user_metrics = lfm4.user_metrics
df = df.merge(user_metrics[user_metrics['subsr_id'].isin(test_df['subsr_id'])][['subsr_id', 'precision']].rename(columns={'precision':'lfm_ver2모델_usertype3_rating4_learning_rate변경'}).sort_values(by='subsr_id'), how='left')


Epoch: 100%|██████████| 10/10 [00:01<00:00,  9.10it/s]


In [ ]:
# 최고모델 - 0.06
model_best = LightFM_Model_ver2(vod_log, cont_log, vod_info, user_type_3, rating_type=1, param_loss='warp', param_components=30, param_epochs=10, param_learning_schedule='adagrad', param_learning_rate=0.05)

Epoch: 100%|██████████| 10/10 [00:02<00:00,  4.19it/s]


In [ ]:
rec_list1 = pd.DataFrame(columns=result_df['subsr'].astype('int')[:-1], index=[f'vod_{x}' for x in range(1, 11)])

In [ ]:
vod_info['program_id'] = vod_info['program_id'].astype('int')

In [ ]:
# 'subsr':	64385000, 'subsr_id':16, model1
user0_rec = model_best.recommend(16, model_best.score_matrix, model_best.model, N=10)

for idx, name in enumerate(vod_info[vod_info['program_id'].isin(user0_rec['program_id'])]['program_name']):
    rec_list1.loc[f'vod_{idx + 1}', 64385000] = name

In [ ]:
# 'subsr': 60593000, 'subsr_id': 22, model1
user1_rec = model_best.recommend(22, model_best.score_matrix, model_best.model, N=10)

for idx, name in enumerate(vod_info[vod_info['program_id'].isin(user1_rec['program_id'])]['program_name']):
    rec_list1.loc[f'vod_{idx + 1}', 60593000] = name

In [ ]:
# 'subsr': 64130000, 'subsr_id': 32, model2
user2_rec = model_best.recommend(32, model_best.score_matrix, model_best.model, N=10)

for idx, name in enumerate(vod_info[vod_info['program_id'].isin(user2_rec['program_id'])]['program_name']):
    rec_list1.loc[f'vod_{idx + 1}', 64130000] = name

In [ ]:
# 'subsr': 64984000, 'subsr_id': 55, model3
user3_rec = model_best.recommend(55, model_best.score_matrix, model_best.model, N=10)

for idx, name in enumerate(vod_info[vod_info['program_id'].isin(user3_rec['program_id'])]['program_name']):
    rec_list1.loc[f'vod_{idx + 1}', 64984000] = name

In [ ]:
# 'subsr': 66056000, 'subsr_id': 89, model4
user4_rec = model_best.recommend(89, model_best.score_matrix, model_best.model, N=10)

for idx, name in enumerate(vod_info[vod_info['program_id'].isin(user4_rec['program_id'])]['program_name']):
    rec_list1.loc[f'vod_{idx + 1}', 66056000] = name

In [ ]:
# 'subsr': 66656000, 'subsr_id': 90, model3
user5_rec = model_best.recommend(90, model_best.score_matrix, model_best.model, N=10)

for idx, name in enumerate(vod_info[vod_info['program_id'].isin(user5_rec['program_id'])]['program_name']):
    rec_list1.loc[f'vod_{idx + 1}', 66656000] = name

In [ ]:
# 'subsr': 66875000, 'subsr_id': 101, model5
user6_rec = model_best.recommend(101, model_best.score_matrix, model_best.model, N=10)

for idx, name in enumerate(vod_info[vod_info['program_id'].isin(user6_rec['program_id'])]['program_name']):
    rec_list1.loc[f'vod_{idx + 1}', 66875000] = name

In [ ]:
# 'subsr': 66705000, 'subsr_id': 125, model6
user7_rec = model_best.recommend(125, model_best.score_matrix, model_best.model, N=10)

for idx, name in enumerate(vod_info[vod_info['program_id'].isin(user7_rec['program_id'])]['program_name']):
    rec_list1.loc[f'vod_{idx + 1}', 66705000] = name

In [ ]:
# 'subsr': 60067000, 'subsr_id': 128, model7
user8_rec = model_best.recommend(128, model_best.score_matrix, model_best.model, N=10)

for idx, name in enumerate(vod_info[vod_info['program_id'].isin(user8_rec['program_id'])]['program_name']):
    rec_list1.loc[f'vod_{idx + 1}', 60067000] = name

In [ ]:
# 'subsr': 62248000, 'subsr_id': 170, model3
user9_rec = model_best.recommend(170, model_best.score_matrix, model_best.model, N=10)

for idx, name in enumerate(vod_info[vod_info['program_id'].isin(user9_rec['program_id'])]['program_name']):
    rec_list1.loc[f'vod_{idx + 1}', 62248000] = name

In [ ]:
result_df.to_csv('/content/drive/MyDrive/DXSCHOOL/Project/performance_pkl/performance_df_1227_unseen만.csv')

In [ ]:
result_df = df.copy()

In [ ]:
result_df.loc['Mean'] = result_df.mean()

In [ ]:
result_df.iloc[-1, 2:].max()

0.06

In [ ]:
result_df

,subsr,subsr_id,SVD모델_rating1_기본파라미터,SVD모델_rating2_기본파라미터,SVD모델_rating3_기본파라미터,SVD모델_rating4_기본파라미터,lfm_ver1모델_rating1_기본파라미터,lfm_ver1모델_rating2_기본파라미터,lfm_ver1모델_rating3_기본파라미터,lfm_ver1모델_rating4_기본파라미터,...,lfm_ver2모델_usertype3_rating3_component증가,lfm_ver2모델_usertype3_rating4_component증가,lfm_ver2모델_usertype3_rating1_loss변경,lfm_ver2모델_usertype3_rating2_loss변경,lfm_ver2모델_usertype3_rating3_loss변경,lfm_ver2모델_usertype3_rating4_loss변경,lfm_ver2모델_usertype3_rating1_learning_rate변경,lfm_ver2모델_usertype3_rating2_learning_rate변경,lfm_ver2모델_usertype3_rating3_learning_rate변경,lfm_ver2모델_usertype3_rating4_learning_rate변경
0,64385000.0,16.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,60593000.0,22.0,0.0,0.00,0.0,0.0,0.10,0.00,0.10,0.00,...,0.10,0.00,0.10,0.00,0.10,0.00,0.10,0.00,0.10,0.00
2,64130000.0,32.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,64984000.0,55.0,0.0,0.00,0.0,0.0,0.10,0.10,0.00,0.10,...,0.00,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10
4,66056000.0,89.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,66656000.0,90.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,66875000.0,101.0,0.0,0.10,0.0,0.0,0.10,0.10,0.20,0.00,...,0.20,0.00,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.20
7,66705000.0,125.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,60067000.0,128.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.10,0.10,0.00,0.10,0.10,0.10,0.00,0.10
9,62248000.0,170.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.10,0.10,0.00,0.10,0.10,0.10,0.00,0.10


In [ ]:
# performance_df.sort_values(by='콜드스타트유저_precision', ascending=False) # 0.021429
# performance_df.sort_values(by='비기너유저_precision', ascending=False) # 0.014545
# performance_df.sort_values(by='스탠다드유저_precision', ascending=False) # 0.030189 # 0.032075
# performance_df.sort_values(by='헤비유저_precision', ascending=False) # 0.021898
performance_df.sort_values(by='Precision@10', ascending=False)

,Model,Precision@10,Recall@10,MAP@10,MAR@10,Diversity,콜드스타트유저_precision,비기너유저_precision,스탠다드유저_precision,헤비유저_precision,콜드스타트유저_recall,비기너유저_recall,스탠다드유저_recall,헤비유저_recall,콜드스타트유저_diversity,비기너유저_diversity,스탠다드유저_diversity,헤비유저_diversity
11,lfm_ver1모델_rating4_loss변경,0.019414,0.067379,0.021320,0.048341,0.633396,0.014286,0.012727,0.022642,0.021898,0.142857,0.046212,0.139774,0.032444,0.627547,0.591994,0.656916,0.591598
9,lfm_ver1모델_rating2_loss변경,0.019048,0.073471,0.023309,0.052937,0.552721,0.014286,0.014545,0.022642,0.020438,0.142857,0.079545,0.125623,0.036675,0.367295,0.555744,0.517947,0.582051
8,lfm_ver1모델_rating1_loss변경,0.018681,0.072393,0.029915,0.052333,0.450879,0.014286,0.012727,0.026415,0.018978,0.142857,0.073485,0.132521,0.034292,0.124015,0.434809,0.331287,0.534961
19,lfm_ver1모델_rating4_learning_schedule변경,0.018681,0.081214,0.022220,0.062859,0.626531,0.017857,0.014545,0.024528,0.018248,0.178571,0.082576,0.153925,0.032641,0.510790,0.652855,0.622306,0.610342
17,lfm_ver1모델_rating2_learning_schedule변경,0.018315,0.066672,0.022992,0.048105,0.613425,0.014286,0.009091,0.024528,0.020438,0.142857,0.053030,0.119428,0.036170,0.388901,0.643933,0.552695,0.594079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,lfm_ver2모델_rating4_learning_rate변경,0.011355,0.034121,0.007530,0.022368,0.363174,0.000000,0.003636,0.022642,0.012409,0.000000,0.020455,0.101234,0.020619,0.207059,0.306974,0.275774,0.396563
31,lfm_ver2모델_rating4_loss변경,0.011355,0.034121,0.007530,0.022368,0.363174,0.000000,0.003636,0.022642,0.012409,0.000000,0.020455,0.101234,0.020619,0.207059,0.306974,0.275774,0.396563
74,lfm_ver2모델_usertype3_rating3_component증가,0.010989,0.051480,0.019673,0.039840,0.486473,0.017857,0.003636,0.015094,0.010949,0.160714,0.024242,0.096055,0.022845,0.240373,0.504291,0.380003,0.490636
78,lfm_ver2모델_usertype3_rating3_component증가,0.010989,0.051480,0.019673,0.039840,0.486473,0.017857,0.003636,0.015094,0.010949,0.160714,0.024242,0.096055,0.022845,0.240373,0.504291,0.380003,0.490636


In [ ]:
performance_df.to_pickle('/content/drive/MyDrive/DXSCHOOL/Project/performance_pkl/performance_df_1226_시리즈물포함.pkl')

### 최종 결과 제출

In [ ]:
subsr_id_testlist = [66875000, 64385000, 66056000, 64130000, 66656000, 66705000, 64984000, 60067000, 60593000, 62248000]

결과 도출

- Precision: 기본 파라미터 0.012 → 파라미터 조정 후 0.019
- Diversity: 기본 0.29 → Meta정보 추가 후 0.6

In [ ]:
result1.sort_values(by='Precision@10', ascending=False).head(20) # lfm_ver1모델_rating4_loss변경
result1.sort_values(by='Recall@10', ascending=False).head(20) # lfm_ver1모델_rating4_learning_schedule변경
result1.sort_values(by='MAP@10', ascending=False).head(20) #lfm_ver1모델_rating2_component증가
result1.sort_values(by='MAR@10', ascending=False).head(20) # lfm_ver1모델_rating4_learning_schedule변경
result1.sort_values(by='Diversity', ascending=False).head(20) #lfm_ver2모델_usertype1_rating4_component증가

result1.sort_values(by='콜드스타트유저_precision', ascending=False).head(20) # 	lfm_ver2모델_rating4_기본파라미터
# result1.sort_values(by='콜드스타트유저_recall', ascending=False).head(20) # lfm_ver2모델_rating4_기본파라미터
# result1.sort_values(by='콜드스타트유저_diversity', ascending=False).head(20) #lfm_ver1모델_rating4_epoch변경
result1.sort_values(by='비기너유저_precision', ascending=False).head(20) # lfm_ver1모델_rating4_learning_schedule변경
# result1.sort_values(by='비기너유저_recall', ascending=False).head(20) # 	lfm_ver1모델_rating1_epoch변경
# result1.sort_values(by='비기너유저_diversity', ascending=False).head(20) # lfm_ver1모델_rating4_learning_schedule변경
result1.sort_values(by='스탠다드유저_precision', ascending=False).head(20) #lfm_ver2모델_rating2_learning_rate변경
# result1.sort_values(by='스탠다드유저_recall', ascending=False).head(20) # lfm_ver2모델_rating2_learning_rate변경
# result1.sort_values(by='스탠다드유저_diversity', ascending=False).head(20) #lfm_ver1모델_rating4_epoch변경
result1.sort_values(by='헤비유저_precision', ascending=False).head(20) #lfm_ver1모델_rating4_loss변경
# result1.sort_values(by='헤비유저_recall', ascending=False).head(20) #lfm_ver1모델_rating2_epoch변경
# result1.sort_values(by='헤비유저_diversity', ascending=False).head(20) #	lfm_ver1모델_rating3_epoch변경

,Model,Precision@10,Recall@10,MAP@10,MAR@10,Diversity,콜드스타트유저_precision,비기너유저_precision,스탠다드유저_precision,헤비유저_precision,콜드스타트유저_recall,비기너유저_recall,스탠다드유저_recall,헤비유저_recall,콜드스타트유저_diversity,비기너유저_diversity,스탠다드유저_diversity,헤비유저_diversity
11,lfm_ver1모델_rating4_loss변경,0.019414,0.067379,0.021320,0.048341,0.633396,0.014286,0.012727,0.022642,0.021898,0.142857,0.046212,0.139774,0.032444,6.275471e-01,0.591994,0.656916,0.591598
17,lfm_ver1모델_rating2_learning_schedule변경,0.018315,0.066672,0.022992,0.048105,0.613425,0.014286,0.009091,0.024528,0.020438,0.142857,0.053030,0.119428,0.036170,3.889007e-01,0.643933,0.552695,0.594079
9,lfm_ver1모델_rating2_loss변경,0.019048,0.073471,0.023309,0.052937,0.552721,0.014286,0.014545,0.022642,0.020438,0.142857,0.079545,0.125623,0.036675,3.672954e-01,0.555744,0.517947,0.582051
8,lfm_ver1모델_rating1_loss변경,0.018681,0.072393,0.029915,0.052333,0.450879,0.014286,0.012727,0.026415,0.018978,0.142857,0.073485,0.132521,0.034292,1.240154e-01,0.434809,0.331287,0.534961
7,lfm_ver1모델_rating4_component증가,0.016117,0.060017,0.015123,0.043280,0.230483,0.014286,0.009091,0.016981,0.018978,0.142857,0.062879,0.077412,0.035208,1.110223e-16,0.179346,0.079575,0.325284
13,lfm_ver1모델_rating2_epoch변경,0.017582,0.069194,0.028515,0.049177,0.527505,0.014286,0.010909,0.022642,0.018978,0.142857,0.057576,0.123536,0.037781,3.479576e-01,0.544020,0.464932,0.549275
16,lfm_ver1모델_rating1_learning_schedule변경,0.017216,0.070765,0.023730,0.054164,0.526610,0.014286,0.009091,0.024528,0.018248,0.142857,0.053030,0.151838,0.031786,2.001795e-01,0.594910,0.400305,0.571895
19,lfm_ver1모델_rating4_learning_schedule변경,0.018681,0.081214,0.022220,0.062859,0.626531,0.017857,0.014545,0.024528,0.018248,0.178571,0.082576,0.153925,0.032641,5.107905e-01,0.652855,0.622306,0.610342
3,lfm_ver1모델_rating4_기본파라미터,0.015751,0.059954,0.014881,0.043274,0.231275,0.014286,0.009091,0.016981,0.018248,0.142857,0.062879,0.077412,0.035083,3.140301e-03,0.180017,0.080474,0.325557
50,lfm_ver2모델_usertype1_rating3_learning_rate변경,0.016484,0.069684,0.015989,0.053868,0.523225,0.014286,0.009091,0.022642,0.017518,0.142857,0.062879,0.133035,0.032953,2.404765e-01,0.479138,0.453082,0.561372
